# Librerías a instalar que no vienen por default
Correrlos en cmd para instalarlos<br>  pip install BeautifulSoup4<br>  pip install requests<br>  pip install selenium<br>

Hay que descargar el web driver de tu navegador

# Importar librerías

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Código

In [ ]:
def check_exists_by_xpath(webdriver, xpath):
    """
        Regresa True si existe el elemento con el xpath dado, sino False
    """
    try:
        webdriver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
def selectOpc(driver,xpathDiv1, xpathDivOpc, pos):
    """
        Selecciona la opción localizada en la posición pos de xpathDivOpc
        driver: driver del navegador
        xpathDiv1:
    """
    #Damos click para que aparezca el div de opciones
    driver.find_element_by_xpath(xpathDiv1).click() 
    #Div de opciones
    opcs = driver.find_element_by_xpath(xpathDivOpc).find_elements_by_tag_name("div") 
    #Damos clic a la opción
    opcs[pos].click()

In [ ]:
def getSizeByTag(driver, xpath, tag):
    """
        Dada un xpath busca todos los elementos con el tag dado y regresa el tamaño
    """
    return len(driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag))


In [ ]:
def doClick(driver, xpath):
    driver.find_element_by_xpath(xpath).click()


In [ ]:
def getElementsByTag(driver, xpath, tag):
    return driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag)

In [ ]:
#Variar Tipo de Juzgado, Delitos-Modalidades,  Criterios , Tipo de resolución
#Checar el comentario del primer for (i1)

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")


divJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[1]'
divOpcJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[2]/div'
divDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[1]'
divOpcDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[2]/div'
divCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[1]'
divOpcCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[2]/div'
divResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[1]'
divOpcResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[2]/div'
btnBuscar = '//*[@id="ContentPlaceHolder1_BtnBuscar"]'
btnCerrarVisor = '//*[@id="modalPDF"]/div/div/div[3]/button'
tabla = '//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'
rowsPath = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody'
visorPDF = '//*[@id="ContentPlaceHolder1_framePDF"]'

# Tipo de juzgado ----------------------------------------------------
selectOpc(driver,divJuzgado,divOpcJuzgado, 0)
t1 = getSizeByTag(driver, divOpcJuzgado, "div")

#Si ya es la versión final hay que cambiar el for para que empiece con 1.

#Empieza en 2 para probar
for i1 in range(2,t1):                                     
    selectOpc(driver,divJuzgado,divOpcJuzgado, i1)   

    #Delitos - Modalidades  ----------------------------------------------  
    selectOpc(driver, divDelitos, divOpcDelitos, 0)        
    t2 = getSizeByTag(driver, divOpcDelitos,"div")

    #Las primeras 7 son Ver todos y opciones vacías
    for i2 in range(7,t2):                           
        selectOpc(driver, divDelitos, divOpcDelitos, i2)        

        #Criterios Específicos ----------------------------------------------      
        selectOpc(driver, divCriterios ,divOpcCriterios , 0)                              
        t3 = getSizeByTag(driver, divOpcCriterios,"div")

        for i3 in range(t3):             
            selectOpc(driver, divCriterios ,divOpcCriterios , i3)               

            #Tipo resolución --------------------------------------------------                    
            selectOpc(driver, divResolucion, divOpcResolucion, 0)
            t4 = getSizeByTag(driver, divOpcResolucion,"div")

            for i4 in range(1, t4):                      
                selectOpc(driver, divResolucion, divOpcResolucion, i4)

                #Hacemos consulta -----------------------------------------------
                doClick(driver, btnBuscar)
                time.sleep(3)
                #Pendiente Checar si ya dio el resultado en lugar de poner 3 segundos    
                                
                if check_exists_by_xpath(driver, tabla):
                    #Hay tabla
                    rows = getElementsByTag(driver, rowsPath, "tr")                    
                    nr = len(rows)
                    for i in range(nr):
                        rows = getElementsByTag(driver, rowsPath, "tr")

                        cols = rows[i].find_elements_by_tag_name("td")
                        nCol = len(cols)
                        
                        for y in range(nCol-1):
                            print(cols[y].get_attribute("innerHTML"))
                                                
                        #Abrir pdf para obtener link                       
                        cols[nCol-1].click()
                        #Esperamos a que cargue
                        time.sleep(3)                                                           
                        print(driver.find_element_by_xpath(visorPDF).get_attribute('src'))
                        #Cerrar el visor
                        doClick(driver, btnCerrarVisor)                        

                        #Boton descargar
                        #//*[@id="download"]
                                            


# Pendiente para recorrer todas las páginas de cierta consulta

In [ ]:

#Variar Tipo de Juzgado, Delitos-Modalidades, Sentido del fallo, Criterios , Tipo de resolución

driver = webdriver.Chrome()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")

# Tipo de juzgado
#Damos click para que aparezca el div de opciones
driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[1]').click()
#opciones juzgado
opcJuz = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[2]/div').find_elements_by_tag_name("div")
opcJuz[0].click() # Para cerrar el div
t1 = len(opcJuz)-1

for i1 in range(1,t1):                           
    #Damos clic a juzgado
    driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[1]').click()   
    #Seleccionamos divs de opciones 
    opcJuz = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[2]/div').find_elements_by_tag_name("div") 
    del opcJuz[0]
    opcJuz[i1].click()


    #Delitos - Modalidades    
    #Activamos para que aparezcan las opciones en el div
    driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[1]').click()
    #Sacamos las opciones
    opcDel = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[2]/div').find_elements_by_tag_name("div")    
    opcDel[0].click() # Para cerrar el div
    for i in range(7):
        del opcDel[0]    
    t2 = len(opcDel)

    for i2 in range(t2):                           
        driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[1]').click()

        opcDel = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[2]/div').find_elements_by_tag_name("div")
        #Borramos Ver todos y opciones vacías    
        for i in range(7):
            del opcDel[0]
        opcDel[i2].click()

        

        #Criterios Específicos
        #Activamos para que aparezcan las opciones en el div
        driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[1]').click()
        #Sacamos las opciones
        opcCrit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[2]/div').find_elements_by_tag_name("div")    
        opcCrit[0].click() # Para cerrar el div
                        
        t5 = len(opcCrit)

        for i5 in range(t5):                           
            #Activamos para que aparezcan las opciones en el div
            driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[1]').click()
            #Sacamos las opciones
            opcCrit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[2]/div').find_elements_by_tag_name("div") 
            #Borramos Ver todos y opciones vacías                        
            opcCrit[i5].click()                


            #Tipo resolución

            #Activamos para que aparezcan las opciones en el div
            driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[1]').click()
            #Sacamos las opciones
            opcRes = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[2]/div').find_elements_by_tag_name("div")    
            opcRes[0].click() # Para cerrar el div
            
            del opcRes[0]    
            t6 = len(opcRes)

            for i6 in range(t6):                      
                #Activamos para que aparezcan las opciones en el div
                driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[1]').click()
                #Sacamos las opciones
                opcRes = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[2]/div').find_elements_by_tag_name("div")
                #Borramos Ver todos y opciones vacías  
                if len(opcRes) > 0:  
                    del opcRes[0]
                opcRes[i6].click()
                driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_BtnBuscar"]').click()      
                time.sleep(3)    
                
                            
                if check_exists_by_xpath(driver,'//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'):
                    #Hay tabla
                    
                    #Checo si hay más de 20 sentencias
                    if check_exists_by_xpath(drive, '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[2]'):                              
                            while True:
                                #tbody de la barra que aparece si hay más de 20 sentencias
                                pags = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr').find_elements_by_tag_name("td")
                                numP= len(pags)
                                borrar = False #Borrar el primero porque tiene puntitos
                                sig = False
                                if numP == 11:
                                    #Primeros 10 y para los sig      
                                    """"""
                                    sig = True
                                else if numP == 12:
                                    #Primero es antes, 10 pags y último para los sig\
                                    borrar= True
                                    sig = True
                                else:                                
                                    td = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[1]')
                                    a =  td.find_elements_by_tag_name("a")
                                    sp= td.find_elements_by_tag_name("span")
                                    if len(a) > 0:
                                        #Son puntitos y estamos en la última
                                        print("es intermedio")
                                        borrar = True
                                    else:                            
                                        #es la primera lista
                                        print("es la primera lista")
                                if borrar:
                                    del pags[0]                                
                                ind = 1
                                if sig:
                                    tam = len(pags)-1 # Le quitamos uno por los puntitos para la siguiente lista
                                else:
                                    tam = len(pags)
                                while ind < tam:
                                    #Checa tabla
                                    rows = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody').find_elements_by_tag_name("tr")
                                    nr = len(rows)
                                    for i in range(nr):
                                        rows = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody').find_elements_by_tag_name("tr")
                                        cols = rows[i].find_elements_by_tag_name("td")
                                        nCol = len(cols)
                                        for y in range(nCol-1):
                                            print(cols[y].get_attribute("innerHTML"))
                                        #Visor del pdf
                                        #//*[@id="ContentPlaceHolder1_framePDF"] 
                                        #Boton cerrar
                                        #//*[@id="modalPDF"]/div/div/div[3]/button
                                        #Display popup                        
                                        cols[nCol-1].click()
                                        #Esperamos a que cargue
                                        time.sleep(3)      
                                        #iframe del pdf                  
                                        print(driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_framePDF"]').get_attribute('outerHTML'))
                                        #Cerrar el visor
                                        driver.find_element_by_xpath('//*[@id="modalPDF"]/div/div/div[3]/button').click()
                                        #Boton descargar
                                        #//*[@id="download"]
                                    if ind < tam:
                                        pags[ind].click()
                                    pags = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr').find_elements_by_tag_name("td")
                                    if borrar:
                                        del pags[0]                                                                    

                                    ind+=1
                                if sig:
                                    #Dar click a los puntitos para lo siguiente
                                    pags[10].click()
                                else:
                                    break

                    else:
                        #Solo hay una tabla
                        #Checa tabla
                        rows = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody').find_elements_by_tag_name("tr")
                        nr = len(rows)
                        for i in range(nr):
                            rows = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody').find_elements_by_tag_name("tr")
                            cols = rows[i].find_elements_by_tag_name("td")
                            nCol = len(cols)
                            for y in range(nCol-1):
                                print(cols[y].get_attribute("innerHTML"))
                            #Visor del pdf
                            #//*[@id="ContentPlaceHolder1_framePDF"] 
                            #Boton cerrar
                            #//*[@id="modalPDF"]/div/div/div[3]/button
                            #Display popup                        
                            cols[nCol-1].click()
                            #Esperamos a que cargue
                            time.sleep(3)      
                            #iframe del pdf                  
                            print(driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_framePDF"]').get_attribute('outerHTML'))
                            #Cerrar el visor
                            driver.find_element_by_xpath('//*[@id="modalPDF"]/div/div/div[3]/button').click()
                            #Boton descargar
                            #//*[@id="download"]
                    
                            

